In [8]:
import json
import query_builder
from pymongo import MongoClient
from tqdm import tqdm
import pandas as pd

# Connect to MongoDB

In [3]:
DB_mongo=query_builder.get_mongo_databases(verbose=True)

Ping Successful! You are connected to MongoDB!
GroceryDB Permissions:
xgen-readAtGroceryDB.CleanedData
xgen-readAtGroceryDB.ProductIngredients


### Reading the CleanedData dataset

In [13]:
#Categories to remove for the GroceryDB analysis
print(query_builder.bad_categories_more_strict)

['non-food', 'multi-items', 'alcohol', 'exempt', 'no-category', 'maybe-bug', 'find-category']


In [9]:
#Fetch Data!
gdb=[]
for p in tqdm(DB_mongo['GroceryDB']['CleanedData'].find(
    {"harmonized single category": {"$nin": query_builder.bad_categories_more_strict}})):
    gdb.append(p)
    pass
gdb = pd.DataFrame(gdb)

50591it [00:17, 2895.82it/s]


In [47]:
gdb

,_id,original_ID,name,store,brand,url,store category,Navbar,store category cleaned str,FPro rank percentile,...,num_additives,num_amount_nodes,num_ingredients,num_marker_nodes,Nutritional_similarity_score_across_stores,fdc_id_list,sim_ingreds_oID_list_from_trees,duplicate_IDs,ignore_duplicate,tree_branch_depth_sum
0,612d009c4c4f3be6d9e174b1,wf_green-mountain-creamery-yoyummy-yogurt-sque...,"Yoyummy Yogurt Squeeze Mixed Berry, 3.5 count",WholeFoods,Green Mountain Creamery™,https://www.wholefoodsmarket.com/product/green...,"[Dairy & Eggs, Yogurt]",[None],"['dairy-eggs', 'yogurt']",0.992063,...,9.0,0.0,8.0,0.0,"[[tg_76201366, 0.19597153805685044], [wf_gogo-...","[[553613, 0.703875968992248, 1.0, 0.7038759689...","[[wm_949767676, 0.2523295680339483, 0, 0.25232...",None,False,0.0
1,612d009c4c4f3be6d9e1756a,wf_stonyfield-yobaby-whole-milk-baby-yogurt-po...,"YoBaby Whole Milk Baby Yogurt Pouches, Peach &...",WholeFoods,Stonyfield,https://www.wholefoodsmarket.com/product/stony...,[None],[None],['none'],0.952381,...,4.0,0.0,10.0,0.0,[[wf_stonyfield-organic-yobaby-whole-milk-baby...,"[[1627388, 0.6301105233296711, None, 0.6301105...",[[wf_stonyfield-organic-stonyfield-organic-org...,[wf_stonyfield-organic-yobaby-whole-milk-baby-...,False,2.0
2,612d009c4c4f3be6d9e176b0,wf_stonyfield-yobaby-whole-milk-baby-yogurt-po...,"YoBaby Whole Milk Baby Yogurt Pouches, Banana ...",WholeFoods,Stonyfield,https://www.wholefoodsmarket.com/product/stony...,[None],[None],['none'],0.952381,...,3.0,0.0,6.0,0.0,[[wf_stonyfield-organic-yobaby-whole-milk-baby...,"[[1219323, 0.6438115250237989, None, 0.6438115...",[[wf_stonyfield-organic-yobaby-whole-milk-baby...,None,False,0.0
3,612d009c4c4f3be6d9e179d0,wf_earths-best-stage-2-organic-pears-baby-food...,"Stage 2 Organic Pears Baby Food, 4 oz",WholeFoods,Earth's Best,https://www.wholefoodsmarket.com/product/earth...,[None],[None],['none'],NaN,...,1.0,0.0,2.0,0.0,[[wf_r-w-knudsen-family-organic-pear-juice-fdf...,"[[1485575, 0.977777777777778, None, 0.97777777...","[[tg_13752449, 0.567960415885582, 0, 0.5679604...",None,False,0.0
4,612d009c4c4f3be6d9e186f2,wf_kite-hill-strawberry-banana-yogurt-tubes-61...,"Strawberry Banana Yogurt Tubes, 14 oz",WholeFoods,Kite Hill,https://products.wholefoodsmarket.com/product/...,"[dairy-eggs, dairy-eggs.dairy-alternatives, da...",[None],"['dairy-eggs', 'dairy-alternatives', 'almond',...",0.769841,...,5.0,0.0,13.0,0.0,[[wf_kite-hill-artisan-almond-milk-strawberry-...,"[[1348263, 0.6837226677782374, None, 0.6837226...",[[wf_kite-hill-artisan-almond-milk-strawberry-...,None,False,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50586,612d009d4c4f3be6d9e25a32,tg_47110256,Hawaiian Sun Mango Pineapple Jam - 10oz,Target,Hawaiian Sun,https://www.target.com/p/hawaiian-sun-mango-pi...,[None],"[Target, Grocery, Pantry, Peanut Butter & Jelly]","['grocery', 'pantry', 'peanut-butter-jelly']",NaN,...,4.0,0.0,2.0,0.0,"[[tg_47110249, 0.0], [wm_10308106, 0.0], [tg_4...","[[1497119, 0.9726027397260276, None, 0.9726027...","[[tg_47110249, 0.5334325920667274, 0, 0.533432...",None,False,0.0
50587,612d009d4c4f3be6d9e25a33,tg_47110257,Hawaiian Sun Lite Guava Jam - 10oz,Target,Hawaiian Sun,https://www.target.com/p/hawaiian-sun-lite-gua...,[None],"[Target, Grocery, Pantry, Peanut Butter & Jelly]","['grocery', 'pantry', 'peanut-butter-jelly']",NaN,...,0.0,0.0,1.0,0.0,None,None,"[[tg_77332753, 1.0, 0, 1.0], [wf_floral-tulips...",None,False,NaN
50588,612d009d4c4f3be6d9e25a34,tg_53847901,Kramer's Jalapeno Pepper Jam - 8oz Jar,Target,Kramer's,https://www.target.com/p/kramer-s-jalapeno-pep...,[None],"[Target, Grocery, Pantry, Peanut Butter & Jelly]","['grocery', 'pantry', 'peanut-butter-jelly']",NaN,...,2.0,0.0,3.0,0.0,"[[tg_53847900, 0.0], [tg_53847896, 0.0], [tg_5...","[[1134811, 0.9555555555555556, None, 0.9555555...","[[tg_53847900, 0.7928402227862125, 0, 0.792840...",None,False,0.0
50589,612d009d4c4f3be6d9e25a35,tg_53847896,Kramer's Cranberry Jalapeno Jam - 8 fl 

### Reading the ProductIngredients dataset

In [37]:
#Categories to remove for the GroceryDB analysis
print(query_builder.bad_categories_more_strict)

['non-food', 'multi-items', 'alcohol', 'exempt', 'no-category', 'maybe-bug', 'find-category']


In [39]:
#Fetch ingredient data!
gdbp=[]
for p in tqdm(DB_mongo['GroceryDB']['ProductIngredients'].find(
    {"harmonized single category": {"$nin": query_builder.bad_categories_more_strict}})):
    gdbp.append(p)
    pass
print('ProductIngredient json downloaded')

42204it [00:07, 5739.59it/s]

ProductIngredient json downloaded


In [40]:
# Function to flatten json
def flatten_ingredients(product_name,original_id,ingredients,parent_order=0,depth=1):
    rows=[]

    for ingredient in ingredients:
        # Calculate distance to first node
        distance_to_root=ingredient.get('order',0)+parent_order

        # Extract ingredient information, calculated distance, depth, and parent_order
        row = {
            'product_name': product_name,
            'original_ID': original_id,
            'ingredient_name': ingredient.get('ingredient_name'),
            'original_text': ingredient.get('original_text'),
            'general_name': ingredient.get('general_name'),
            'ingredient_type': ingredient.get('ingredient_type'),
            'order': ingredient.get('order'),
            'descriptors': ingredient.get('descriptors', []),
            'parent_order': parent_order,
            'distance_to_root': distance_to_root,
            'depth': depth
        }
        rows.append(row)

        # Rerun function for sub-ingredients
        if ingredient.get('sub_ingredients'):
            rows.extend(flatten_ingredients(product_name,original_id,
                ingredient['sub_ingredients'],parent_order=distance_to_root,depth=depth+1
            ))

    return rows

In [44]:
#Convert json into Dataframe
all_rows=[]
for product in gdbp:
    product_name=product.get('product_name')
    original_id=product.get('original_ID') 
    if product_name and product.get('ingredient_tree'):  
        all_rows.extend(flatten_ingredients(product_name,original_id,product['ingredient_tree']))

p_ingreds=pd.DataFrame(all_rows)
print('Count of all ingredients in all products: ', len(p_ingreds))

Count of all ingredients in all products:  607072


In [49]:
p_ingreds

,product_name,original_ID,ingredient_name,original_text,general_name,ingredient_type,order,descriptors,parent_order,distance_to_root,depth
0,Organic Salsa Sensation Microgreens Salad,wf_farming-turtles-inc-organic-salsa-sensation...,cilantro,cilantro,cilantro,additive,1,[],0,1,1
1,Organic Salsa Sensation Microgreens Salad,wf_farming-turtles-inc-organic-salsa-sensation...,parsley,parsley,parsley,additive,2,[],0,2,1
2,Organic Salsa Sensation Microgreens Salad,wf_farming-turtles-inc-organic-salsa-sensation...,arugula,arugula,arugula,ingredient,3,[],0,3,1
3,Organic Salsa Sensation Microgreens Salad,wf_farming-turtles-inc-organic-salsa-sensation...,mizuna,mizuna,mizuna,ingredient,4,[],0,4,1
4,Organic Salsa Sensation Microgreens Salad,wf_farming-turtles-inc-organic-salsa-sensation...,snow pea tendril,snow pea tendril,snow pea tendril,ingredient,5,[],0,5,1
...,...,...,...,...,...,...,...,...,...,...,...
607067,Tasty Bite Gluten Free and Vegetarian 1 Step J...,tg_14775310,spice,spice,spice,ingredient,14,[],0,14,1
607068,Tasty Bite Gluten Free and Vegetarian 1 Step J...,tg_14775310,chili pepper,chili pepper,chili pepper,additive,15,[],0,15,1
607069,Tasty Bite Gluten Free and Vegetarian 1 Step J...,tg_14775310,ginger,ginger,ginger,additive,16,[],0,16,1
607070,Tasty Bite Gluten Free and Vegetarian 1 Step J...,tg_14775310,garlic,garlic,garlic,additive,17,[],0,17,1
